# Estimation prix occasion véhicule

Dans le rapport, expliciter la partie métier du use case : qu'est-ce qui a de l'importance ? Problèmes rencontrés ? Choix métiers ? Pourquoi ces choix techniques ? Pourquoi ces algos ? les essais réalisés ?

Prix véhicule neuf (amener nouvelles données, autoplus.fr), options, chevaux vapeur, finition, etc)

## Import

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nltk import word_tokenize
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
#from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

import re

pd.set_option('display.max_colwidth', 200)
TRIGGER_VISUALIZATION = False

## Dataset(s)

In [33]:
cars_data = 'mergedAutoplus.csv'
#cars_data = 'Data_cars.csv'


df = pd.read_csv(cars_data)
display(df.shape,
        df.head())

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(166695, 18)

,Price,Make,Model,Model_year,Mileage,Fuel,Gearbox,Online,Description,Make_Model,BOITE_VITESSE,DATE,ENERGIE,MALUS,MARQUE_MODELE,PRIX_NEUF,PUISS_FISCALE,VERSION
0,11220.0,PEUGEOT,308,2014,94341.0 km,Diesel,mécanique,06/07/2018 à 3h47,"modele: 308 (2E GENERATION), version: 1.6 HDI FAP 92CH BUSINESS 5P, puissance_fiscale: 4, portes: 5.0, options: banquette arrière 3 places;volant cuir;cache bagages;airbag frontal;banquette 1/3 - ...",PEUGEOT 308,Manuelle,2018,Diesel,0,Peugeot 308,23500,5CV,1.5 BlueHDi 100 BVM6 Access (5p.)
1,57526.0,BMW,X6,2015,39051.0 km,Diesel,automatique,03/04/2018 à 16h41,"modele: X6 F16, version: (F16) XDRIVE30D 258 M SPORT BVA8, puissance_fiscale: 16, portes: 5.0, options: triangle de presignalisation et trousse de premiers secours;volant cuir;pack alu;affichage t...",BMW X6,Automatique,2019,Diesel,7462,BMW X6,79100,16CV,xDrive30d 265ch BVA8 Lounge (Tout-Terrain)
2,80379.0,AUDI,RS6,2014,75381.0 km,Essence,automatique,30/07/2018 à 1h55,"modele: RS6 (3E GENERATION) AVANT, version: III AVANT 4.0 TFSI 560 QUATTRO TIPTRONIC, puissance_fiscale: 47, portes: 5.0, options: banquette arrière 3 places;écran tactile;témoin de bouclage des c...",AUDI RS6,Automatique,2019,Essence,450,Audi A4,35800,8CV,35 TFSI 150 S tronic 7 (4p.)
3,2830.0,FORD,KA,2007,92282.0 km,Essence,mécanique,09/07/2018 à 14h12,"modele: KA, version: 1.3 70 FUN, puissance_fiscale: 5, portes: 3.0, options: direction assistée;jantes alu;rétroviseurs électriques et dégivrants;vitres électriques;fermeture électrique;climatisat...",FORD KA,Manuelle,2018,Essence,210,Ford Ka +,10450,4CV,1.2i 70 Essential (5p.)
4,12925.0,FIAT,TIPO,2018,10.0 km,Essence,mécanique,19/05/2018 à 3h52,"modele: TIPO 2 SW, version: II SW 1.4 95 POP, puissance_fiscale: 5, portes: 5.0, options: Volant alu & cuir;Bluetooth inclut musique en streaming, connexion téléphone;Rétroviseurs extérieurs chauf...",FIAT TIPO,Manuelle,2019,Essence,740,Fiat Tipo,13490,5CV,1.4 95 (4p.)


## Extraction des descriptions

In [34]:
df = pd.concat([df, df['Description'].str.extract(
    'modele: (?P<Modele>.*?), version: (?P<Version>.*?), puissance_fiscale: (?P<Puissance_fiscale>.*?), portes: (?P<Portes>.*?), options: (?P<Options>.*?), couleur: (?P<Couleur>.*?$)'
)], axis=1, sort=False)
df.head(1)

,Price,Make,Model,Model_year,Mileage,Fuel,Gearbox,Online,Description,Make_Model,...,MARQUE_MODELE,PRIX_NEUF,PUISS_FISCALE,VERSION,Modele,Version,Puissance_fiscale,Portes,Options,Couleur
0,11220.0,PEUGEOT,308,2014,94341.0 km,Diesel,mécanique,06/07/2018 à 3h47,"modele: 308 (2E GENERATION), version: 1.6 HDI FAP 92CH BUSINESS 5P, puissance_fiscale: 4, portes: 5.0, options: banquette arrière 3 places;volant cuir;cache bagages;airbag frontal;banquette 1/3 - ...",PEUGEOT 308,...,Peugeot 308,23500,5CV,1.5 BlueHDi 100 BVM6 Access (5p.),308 (2E GENERATION),1.6 HDI FAP 92CH BUSINESS 5P,4,5.0,banquette arrière 3 places;volant cuir;cache bagages;airbag frontal;banquette 1/3 - 2/3;sièges rang 2 rabattables à plat;vitres teintées;boucliers av et ar couleur caisse;phares av. de jour à LED;...,BLANC BANQUISE


## Conversion en donnée numérique et Nettoyage

In [35]:
df.Model_year = df.Model_year.astype('int64')
df.Mileage = df.Mileage.str.replace(' km', '').astype('float64')
df.Puissance_fiscale = df.Puissance_fiscale.astype('int64')
df.PUISS_FISCALE = df.PUISS_FISCALE.str.replace('CV', '').astype('int64')
df.Portes = df.Portes.replace('', 0).astype('float64')
df.PRIX_NEUF = df.PRIX_NEUF.astype('int64')
df.dtypes

Price                float64
Make                  object
Model                 object
Model_year             int64
Mileage              float64
Fuel                  object
Gearbox               object
Online                object
Description           object
Make_Model            object
BOITE_VITESSE         object
DATE                   int64
ENERGIE               object
MALUS                 object
MARQUE_MODELE         object
PRIX_NEUF              int64
PUISS_FISCALE          int64
VERSION               object
Modele                object
Version               object
Puissance_fiscale      int64
Portes               float64
Options               object
Couleur               object
dtype: object

### Exploration 

In [36]:
df[['Make', 'Price']].groupby(['Make']).mean().head()

,Price
Make,
ABARTH,20481.254237
AC,87696.000000
AIXAM,8980.285714
ALFA ROMEO,20088.910834
ALPINA,53933.800000


### Prix suivant la marque
(Prix plafonnées à 200 000€)

In [37]:
if TRIGGER_VISUALIZATION:
    fig = plt.figure(figsize=(100,10))

    ax1 = sns.violinplot('Make', "Price", data=df[df.Price < 200000])
    ax1.minorticks_on()
    ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax1.grid(which='minor', axis='x', linewidth=1)


### Prix suivant l'année de sortie
(Prix plafonnées à 200 000€)

In [38]:
if TRIGGER_VISUALIZATION:
    fig = plt.figure(figsize=(100,10))

    ax1 = sns.violinplot('Model_year', "Price", data=df[df.Price < 200000])
    ax1.minorticks_on()
    ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax1.grid(which='minor', axis='x', linewidth=1)

### Kilométrage suivant la marque
(Prix plafonnées à 200 000€)

In [39]:
if TRIGGER_VISUALIZATION:
    fig = plt.figure(figsize=(100,10))

    ax1 = sns.violinplot('Make', "Mileage", data=df[df.Price < 200000])
    ax1.minorticks_on()
    ax1.xaxis.set_minor_locator(AutoMinorLocator(2))
    ax1.grid(which='minor', axis='x', linewidth=1)

### Extraction des informations de version
Extraire cylindrée et Chevaux Vapeur

In [40]:
#Chevaux Vapeur
Chevaux = df.Version.str.extract('(?P<cylindree>\d{2,3})')['cylindree'].astype("float64")
df['Chevaux'] = Chevaux

In [41]:
#Cylindree
CylindreeRaw = df.Version.str.extract('(?P<Cylindree1>\d{1}).(?P<Cylindree2>\d{1})')
Cylindree = (CylindreeRaw['Cylindree1'] + '.' + CylindreeRaw['Cylindree2']).astype('float64')
df['Cylindree'] = Cylindree

### Extraction de l'Age du véhicule au moment de la vente
année de mise en ligne de l'annonce - année de sortie

In [42]:
Online_year = df.Online.str.extract('(?P<Jour>\d{2})/(?P<Mois>\d{2})/(?P<Annee>\d{4})')['Annee'].astype('int64')
df['Model_age'] = Online_year - df['Model_year']

### Get Dummies (Fuel, Gearbox, Make, Model)

In [43]:
def apply_dummies(x,df):
    temp = pd.get_dummies(df[x], drop_first = True)
    df = pd.concat([df, temp], axis = 1)
    df.drop([x], axis = 1, inplace = True)
    return df

convert = LabelEncoder()

def apply_label_encoder(label, dataframe):
    return convert.fit_transform(dataframe[label].tolist())
    

In [44]:
# Fuel
#df['Fuel'] = apply_label_encoder('Fuel', df)
df = apply_dummies('Fuel', df)

In [45]:
# Gearbox
#df['Gearbox'] = apply_label_encoder('Gearbox', df)
df = apply_dummies('Gearbox', df)

In [46]:
# Make
#df['Make'] = apply_label_encoder('Make', df)
df = apply_dummies('Make', df)

In [47]:
# Model
#df['Model'] = apply_label_encoder('Model', df)
df = apply_dummies('Model', df)

In [48]:
# Modele
#df['Modele'] = apply_label_encoder('Modele', df)
df = apply_dummies('Modele', df)

## Feature Selection

In [49]:
dfX = df.copy()
dfX.drop(['Price', 'Description', 'Version', 
          'Options', 'Couleur', 'Online', 'Make_Model', 
          'BOITE_VITESSE', 'DATE', 'ENERGIE', 'MALUS', 'MARQUE_MODELE', 'VERSION',
          'PUISS_FISCALE', 'PRIX_NEUF'], axis = 1, inplace = True)
dfX['Chevaux'] = dfX['Chevaux'].fillna(df['Chevaux'].mean())
dfX['Cylindree'] = dfX['Cylindree'].fillna(df['Cylindree'].mean())

dfY = df['Price']

X_train, X_test, y_train, y_test = train_test_split(dfX, dfY, test_size=0.3, random_state=42);

In [51]:
#display(dfX[dfX.isna().any(axis=1)], dfX.head())
from sklearn.ensemble import RandomForestRegressor


# Prevalidate Model
Regressor = RandomForestRegressor(n_jobs=-1, random_state=84, bootstrap= True, max_features= 'sqrt', min_samples_split= 4, n_estimators= 100)


scores = cross_validate(Regressor, X_test, y_test, n_jobs=-1, cv=5, return_estimator=True)
scores

{'fit_time': array([19.71785522, 18.90506792, 18.87693214, 18.93401718, 18.85877609]),
 'score_time': array([0.24331903, 0.24845695, 0.24790454, 0.24776602, 0.25449443]),
 'estimator': (RandomForestRegressor(max_features='sqrt', min_samples_split=4, n_jobs=-1,
                        random_state=84),
  RandomForestRegressor(max_features='sqrt', min_samples_split=4, n_jobs=-1,
                        random_state=84),
  RandomForestRegressor(max_features='sqrt', min_samples_split=4, n_jobs=-1,
                        random_state=84),
  RandomForestRegressor(max_features='sqrt', min_samples_split=4, n_jobs=-1,
                        random_state=84),
  RandomForestRegressor(max_features='sqrt', min_samples_split=4, n_jobs=-1,
                        random_state=84)),
 'test_score': array([0.90710268, 0.8414272 , 0.90047759, 0.92419178, 0.93029362])}

In [59]:
best_estimator = scores['estimator'][-1:][0]

## Model Selection
Faire un model simple et mesurer sa perf avec MAPE (Mean absolute percentage error)
One hot encoding ou
dummyfication car il n'y a que 6 variables différentes dans la colonne fuel

Pour matcher prix neuf d'un autre dataset, on concatene make et model et on compare avec make model du dataset autoplus.fr. Comparaison avec `fuzz.token_set_ratio`.

Voir dataframe mapper : sklearn_pandas DataFrameMapper, sklearn.pipeline Pipeline, sklearn.decomposition PCA

### Utiliser décomposition PCA pour réduire le nombre de colonnes / dimensions (éviter plus de 1000/2000 colonnes --> dispersion de l'information

Parser options: tf idf, traitement fréquentiel.


In [25]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
#import xgboost as xgb

#Regressor = AdaBoostRegressor()
Regressor = RandomForestRegressor(n_jobs=-1, random_state=84, bootstrap= True, max_features= 'sqrt', min_samples_split= 4, n_estimators= 100)
Regressor.fit(X_train, y_train)

#xgtrain = xgb.DMatrix(X_train.values, y_train.values)
#xgtest = xgb.DMatrix(X_test.values)

#param = {'max_depth':2, 'eta':1 }
#num_round = 2
#bst = xgb.train(param, xgtrain, num_round)

RandomForestRegressor(max_features='sqrt', min_samples_split=4, n_jobs=-1,
                      random_state=84)

In [60]:
y_pred = best_estimator.predict(X_test)
#y_pred = bst.predict(xgtest)

## MAPE

In [61]:
import numpy as np
import math

def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [62]:
mean_absolute_error(y_test, y_pred, sample_weight=None, multioutput='uniform_average')

1123.007749451799

In [63]:
print("Mean Percentage Error: {}%".format(
    math.floor(
        mean_absolute_percentage_error(y_test, y_pred)*100
    ) /100
))

Mean Percentage Error: 12.5%


In [64]:
display(y_pred, y_test.values)

array([ 8026.72855952,  8444.4207381 , 11440.1127381 , ...,
       26495.4213403 , 12756.22257143, 19242.91077381])

array([ 7942.,  8936., 11651., ..., 25938., 12719., 19161.])

## GridSearch Test

In [21]:
from time import time
from sklearn.ensemble import RandomForestRegressor


estimator = RandomForestRegressor(random_state = 84, n_jobs=-1)
param_grid = { 
            "n_estimators"      : [30,50,100],
            "max_features"      : ["auto", "sqrt", "log2"],
            "min_samples_split" : [2,4,8],
            "bootstrap": [True, False],
            }

grid_search = GridSearchCV(estimator, param_grid, n_jobs=-1)
    
grid_search.fit(X_train, y_train)
print(grid_search.best_score_ , grid_search.best_params_)

0.821022876629978 {'bootstrap': True, 'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 100}


Meilleur score:
0.821022876629978 {'bootstrap': True, 'max_features': 'sqrt', 'min_samples_split': 4, 'n_estimators': 100}

## (OPTIONNEL) TF IDF and language feature extraction

In [264]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(df['Options'])

In [290]:
v.get_feature_names()
v.vocabulary_

{'banquette': 4264,
 'arrière': 3803,
 'places': 11698,
 'volant': 16176,
 'cuir': 6127,
 'cache': 4840,
 'bagages': 4223,
 'airbag': 3235,
 'frontal': 8006,
 'sièges': 14376,
 'rang': 13203,
 'rabattables': 13123,
 'plat': 11728,
 'vitres': 16134,
 'teintées': 15199,
 'boucliers': 4612,
 'av': 4097,
 'et': 7428,
 'ar': 3708,
 'couleur': 5961,
 'caisse': 4859,
 'phares': 11584,
 'de': 6267,
 'jour': 9238,
 'led': 9520,
 'airbags': 3240,
 'rideaux': 13593,
 'rétroviseurs': 13906,
 'électriques': 16548,
 'dégivrants': 6820,
 'siège': 14373,
 'conducteur': 5737,
 'avec': 4124,
 'réglage': 13842,
 'lombaire': 9725,
 'feux': 7722,
 'freinage': 7989,
 'urgence': 15838,
 'rabattable': 13122,
 'afu': 3189,
 'contrôle': 5874,
 'pression': 11984,
 'des': 6372,
 'pneus': 11795,
 'front': 8004,
 'lat': 9449,
 'blanc': 4453,
 'banquise': 4266,
 'lampes': 9409,
 'lecture': 9518,
 'réglables': 13840,
 'en': 7219,
 'hauteur': 8493,
 'alarme': 3292,
 'oubli': 11135,
 'extinction': 7558,
 'ebd': 6951,
 

In [291]:
from sklearn.feature_extraction.text import CountVectorizer

cvec = CountVectorizer(min_df=1, max_df=.5, ngram_range=(2,3))
cvec.fit(df['Options'])

CountVectorizer(max_df=0.5, ngram_range=(2, 3))

In [298]:
#cvec.get_feature_names()
cvec.vocabulary_

{'banquette arrière': 106627,
 'arrière places': 68485,
 'places volant': 427683,
 'volant cuir': 582136,
 'cuir cache': 186684,
 'cache bagages': 131444,
 'bagages airbag': 104994,
 'airbag frontal': 36629,
 'frontal banquette': 277884,
 'banquette sièges': 107893,
 'sièges rang': 508348,
 'rang rabattables': 466702,
 'rabattables plat': 463771,
 'plat vitres': 428766,
 'vitres teintées': 578652,
 'teintées boucliers': 541294,
 'boucliers av': 124301,
 'av et': 92604,
 'et ar': 250100,
 'ar couleur': 60386,
 'couleur caisse': 179371,
 'caisse phares': 132419,
 'jour led': 325410,
 'led airbags': 338451,
 'airbags rideaux': 38840,
 'rideaux rétroviseurs': 476424,
 'électriques et': 602256,
 'et dégivrants': 252254,
 'dégivrants siège': 228957,
 'siège conducteur': 505882,
 'conducteur avec': 169321,
 'avec réglage': 102323,
 'réglage lombaire': 488006,
 'lombaire vitres': 350623,
 'électriques feux': 602317,
 'feux de': 267896,
 'de freinage': 199566,
 'freinage urgence': 276539,
 'urg